<a href="https://colab.research.google.com/github/freddysundowner/Flutter/blob/develop/ShambaKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LOAD LIBS

In [1]:
!pip install -q opendatasets
import opendatasets as od
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

LOAD DATA

In [2]:
od.download('https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset') # insert ypu kaggle  username and key

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alutatechnologies
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset


100%|██████████| 2.70G/2.70G [00:27<00:00, 104MB/s]


In [3]:
train = "/content/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train"
valid = "/content/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

In [7]:
classes_length = len(os.listdir(train))
classes_length

38

In [8]:
train_datagen = ImageDataGenerator(zoom_range=0.5, shear_range=0.3,horizontal_flip= True, preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [9]:
train = train_datagen.flow_from_directory(
    directory= train,
    target_size = (256,256),
    batch_size = 32)
val = val_datagen.flow_from_directory(
    directory= valid,
    target_size = (256,256),
    batch_size = 32)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [10]:
from keras.layers import Dense, Flatten
from keras.models import Model
import keras

In [11]:
base_model = VGG19(input_shape=(256,256,3), include_top= False)

80134624/80134624 [==============================] - 0s 0us/step


In [12]:
for layer in base_model.layers:
  layer.trainable = False

In [13]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [14]:
x = Flatten()(base_model.output)
x = Dense(units = classes_length, activation = 'softmax')(x)
model = Model(base_model.input, x)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
es = EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 0.01,
    patience = 3,
    verbose = 1
)

mc = ModelCheckpoint(
        filepath = "best_model.h5",
        monitor = "val_accuracy",
        min_delta = 0.01,
        patience = 3,
        verbose = 1,
        save_best_only = True
)
cb = [es,mc]

In [17]:
model.compile(
    optimizer="adam",
    loss = keras.losses.categorical_crossentropy,
    metrics = ['accuracy']
    )

In [18]:
his = model.fit(train,
                                steps_per_epoch =16,
                                epochs = 50,
                                verbose = 1,
                                callbacks = cb,
                                validation_data = val,
                                validation_steps = 16
      )

Epoch 1/50


KeyboardInterrupt: 

test model

In [ ]:
from keras.models import load_model
model = load_model("/content/best_model.h5")

In [ ]:
def prediction(img):
    img = load_img(img, target_size=(256,256))  # Resize the image to match the model's expected input shape
    i = img_to_array(img)
    im = preprocess_input(i)
    img = np.expand_dims(im, axis=0)
    pred = np.argmax(model.predict(img))
    print(pred)

In [ ]:
path = "/content/new-plant-diseases-dataset/test/test/AppleCedarRust2.JPG"
prediction(path)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed